---
title: Searching by region
description: Demonstration of how to find flight segments by geographic region
date: 2025-08-07
---

This notebook demonstrates how to query for radar frames within a particular geometric area.

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import xarray as xr
import hvplot.xarray
import geoviews as gv
import geoviews.feature as gf
import cartopy.crs as ccrs
import matplotlib.pyplot as plt

import xopr.opr_access
import xopr.geometry

hvplot.extension('bokeh')

# Useful projections
epsg_3031 = ccrs.Stereographic(central_latitude=-90, true_scale_latitude=-71)
latlng = ccrs.PlateCarree()
features = gf.ocean.options(scale='50m').opts(projection=epsg_3031) * gf.coastline.options(scale='50m').opts(projection=epsg_3031)

In [ ]:
# Establish an OPR session
# You'll probably want to set a cache directory if you're running this locally to speed
# up subsequent requests. You can do other things like customize the STAC API endpoint,
# but you shouldn't need to do that for most use cases.
opr = xopr.opr_access.OPRConnection(cache_dir="radar_cache")

# Or you can open a connection without a cache directory (for example, if you're parallelizing
# this on a cloud cluster without persistent storage).
#opr = xopr.OPRConnection()

In [ ]:
region = xopr.geometry.get_antarctic_regions(name="Dotson", type="FL", merge_regions=True)
region

In [ ]:
frames = opr.search_by_geometry(region, max_items=10) # To speed things up for this example, we limit the number of items returned to 10.

In [ ]:
# Plot a map of our loaded data over the selected region on an EPSG:3031 projection

# Create a GeoViews object for the selected region
region_gv = gv.Polygons(region, crs=latlng).opts(
    color='green',
    line_color='black',
    fill_alpha=0.5,
    projection=epsg_3031,
)
# Plot the frame geometries
frame_lines = []
for frame in frames:
    coords = frame[['Longitude', 'Latitude']].to_dataframe().dropna()
    line_plot = gv.Path([coords.values]).opts()
    frame_lines.append(line_plot)

(features * region_gv * gv.Overlay(frame_lines)).opts(projection=epsg_3031)